task name: Resolve NaiveBayes Task
task description: Implement the NaiveBayes algorithm using the provided C# code and save the output to a file.

In [ ]:
#r "nuget:Microsoft.ML,1.5.5"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
private class DataPoint
{
    public uint Label { get; set; }
    [VectorType(20)]
    public float[] Features { get; set; }
}

private static IEnumerable<DataPoint> GenerateRandomDataPoints(int count,
    int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)(random.NextDouble() - 0.5);
    for (int i = 0; i < count; i++)
    {
        // Generate Labels that are integers 1, 2 or 3
        var label = random.Next(1, 4);
        yield return new DataPoint
        {
            Label = (uint)label,
            // Create random features that are correlated with the label.
            // The feature values are slightly increased by adding a
            // constant multiple of label.
            Features = Enumerable.Repeat(label, 20)
                .Select(x => randomFloat() + label * 0.2f).ToArray()

        };
    }
}

var dataPoints = GenerateRandomDataPoints(1000);
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

In [ ]:
var pipeline =
    mlContext.Transforms.Conversion
    .MapValueToKey(nameof(DataPoint.Label))
    .Append(mlContext.MulticlassClassification.Trainers.NaiveBayes());

var model = pipeline.Fit(trainingData);

In [ ]:
private class Prediction
{
    // Original label.
    public uint Label { get; set; }
    // Predicted label from the trainer.
    public uint PredictedLabel { get; set; }
}

var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(500, seed: 123));

var transformedTestData = model.Transform(testData);

var predictions = mlContext.Data.CreateEnumerable<Prediction>(transformedTestData, reuseRowObject: false).ToList();

In [ ]:
public static void PrintMetrics(MulticlassClassificationMetrics metrics)
{
    Console.WriteLine($"Micro Accuracy: {metrics.MicroAccuracy:F2}");
    Console.WriteLine($"Macro Accuracy: {metrics.MacroAccuracy:F2}");
    Console.WriteLine($"Log Loss: {metrics.LogLoss:F2}");
    Console.WriteLine($"Log Loss Reduction: {metrics.LogLossReduction:F2}\n");

    Console.WriteLine(metrics.ConfusionMatrix.GetFormattedConfusionTable());
}

var metrics = mlContext.MulticlassClassification.Evaluate(transformedTestData);

PrintMetrics(metrics);